This is using ElevenLabs, Twillio; Need to buld Gemini agent on top of this

In [1]:
!pip install twilio requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.1 MB/s eta 0:00:00


In [ ]:
import os
from twilio.twiml.voice_response import VoiceResponse, Say
from twilio.rest import Client
import requests

# ---------------------- Configuration (Use Environment Variables) ----------------------
TWILIO_ACCOUNT_SID = os.environ.get("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.environ.get("TWILIO_AUTH_TOKEN")
TWILIO_PHONE_NUMBER = os.environ.get("TWILIO_PHONE_NUMBER")  # Your Twilio phone number
ELEVENLABS_API_KEY = os.environ.get("ELEVENLABS_API_KEY")

# ---------------------- Configuration ----------------------
TARGET_PHONE_NUMBER = "+14242704893" # SID UMBER
TARGET_NAME = "Rohan Roy"  
ELEVENLABS_VOICE_ID = "pNInz6obpgDQGcFmaJgB"  # Example voice ID, replace with your desired voice


# ---------------------- ElevenLabs Text-to-Speech Function ----------------------
def text_to_speech(text, voice_id=ELEVENLABS_VOICE_ID):
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    headers = {
        "xi-api-key": ELEVENLABS_API_KEY,
        "Content-Type": "application/json",
        "Accept": "audio/mpeg"  # Request audio in MP3 format
    }
    data = {
        "text": text,
        "model_id": "eleven_monolingual_v1", # Or another model if needed
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.5
        }
    }

    try:
        response = requests.post(url, json=data, headers=headers, stream=True)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        # Save the audio to a file (optional, but useful for Twilio)
        audio_file_path = "output.mp3"  # Or another name and location
        with open(audio_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        return audio_file_path # Return the path to the audio file
    except requests.exceptions.RequestException as e:
        print(f"ElevenLabs API Error: {e}")
        return None


# ---------------------- Twilio Call Handling ----------------------
def make_call(target_number, tts_text):
    """
    Makes an outbound call to the specified target number and plays the TTS message.
    """
    client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

    # Generate speech from ElevenLabs
    audio_file = text_to_speech(tts_text)

    if not audio_file:
        print("Failed to generate audio from ElevenLabs.")
        return

    # Construct the TwiML to play the audio.  We use a simple approach of hosting the audio file.
    #  Alternative:  You could generate TwiML dynamically with <Say> and the ElevenLabs TTS API,
    #  but that is generally more complex.
    twiml = f"""
    <Response>
        <Play>{audio_file}</Play>
    </Response>
    """

    try:
        call = client.calls.create(
            to=target_number,
            from_=TWILIO_PHONE_NUMBER,
            twiml=twiml, # This is the crucial part that tells Twilio what to do.
            # url="YOUR_SERVER_URL/twilio_handler"  # Alternative:  If you want to use a dynamic TwiML generator.
        )
        print(f"Call SID: {call.sid}")
    except Exception as e:
        print(f"Twilio Call Error: {e}")



# ---------------------- Main Execution ----------------------
if __name__ == "__main__":
    message_to_say = "Hello, this is a test call from ElevenLabs and Twilio."
    make_call(TARGET_PHONE_NUMBER, message_to_say)
    print("Outbound call initiated.")
